In [1]:
#default_exp train

In [2]:
#export
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *

Loading imports


In [3]:
torch.cuda.set_device(3)

In [4]:
#export
def get_pil_fn(p):
    def _f(fn): return PILCTScan.create(p/f'{fn}.jpg')
    return _f

In [5]:
#export
def fn2label(fn): return Meta.df_comb.loc[fn][htypes].values.astype(np.float32)

In [6]:
#export
mean = [x/255.0 for x in [ 56.2214,  62.1220, 141.9133]]
std = [x/255.0 for x in [93.0132, 77.8876, 50.8730]]

In [7]:
#export
def get_wgts(df, splits):
    wgts = df['any'][splits[0]].values
    return wgts * (1/0.14 - 2) + 1

In [8]:
#export
def get_data_gen(fns, bs, img_tfm, splits, sz=None, nw=8, mean=mean, std=std,
        wgts=None, batch_xtra=None, after_item=None, with_aug=True, test=False, **kwargs):
    tfms = [[img_tfm, ToTensor], [fn2label,EncodedMultiCategorize(htypes)]]
    if test: tfms = [tfms[0]]
    dsrc = DataSource(fns, tfms, splits=splits)
    nrm = Normalize.from_stats(mean,std)
    batch_tfms = L(IntToFloatTensor, nrm, Cuda()) + L(batch_xtra)
    if with_aug: batch_tfms += aug_transforms(**kwargs)
    if sz is not None:
        batch_tfms = batch_tfms+[RandomResizedCropGPU(sz, min_scale=0.7, ratio=(1.,1.), valid_scale=0.9)]
    if wgts is None:
        return dsrc.databunch(bs=bs, num_workers=nw, after_item=after_item, after_batch=batch_tfms)
    else:
        return dsrc.weighted_databunch(wgts, bs=bs, num_workers=nw, after_item=after_item, after_batch=batch_tfms)


In [9]:
#export
def filename(o): return os.path.splitext(os.path.basename(o))[0]

In [28]:
#export
def get_data(bs, sz, img_dir='nocrop_jpg', full_ds=False):
    if sz is None or sz <= 256: img_dir = f'{img_dir}256'
    df,splits = (Meta.df_comb,Meta.splits) if full_ds else (Meta.df_any,Meta.splits_any)
    return get_data_gen(L(list(df.index)), bs=bs, img_tfm=get_pil_fn(path/img_dir), 
                        sz=sz, splits=splits)

## Training

In [29]:
#export
def accuracy_any(inp, targ, thresh=0.5, sigmoid=True):
    inp,targ = flatten_check(inp[:,0],targ[:,0])
    if sigmoid: inp = inp.sigmoid()
    return ((inp>thresh)==targ.bool()).float().mean()


def get_loss(scale=None):
    num_classes = 6
    loss_weights = to_device(tensor(2.0, 1, 1, 1, 1, 1))
    loss_weights = loss_weights/loss_weights.sum()*num_classes
    
    if scale is not None: scale = to_device(tensor([scale]*num_classes))
    return BaseLoss(nn.BCEWithLogitsLoss, weight=loss_weights, #pos_weight=scale,
                    floatify=True, flatten=False, is_2d=False, activation=torch.sigmoid)


In [30]:
#export
def get_learner(dbch, arch_or_model, lf=None, pretrained=False, opt_func=None, metrics=None, fp16=True, config=None):
    if lf is None: lf = get_loss()
    if metrics is None: metrics=[accuracy_multi,accuracy_any]
    if opt_func is None: opt_func = partial(Adam, wd=1e-5, eps=1e-4, sqr_mom=0.999)
    if isinstance(arch_or_model, nn.Module):
        learn = Learner(dbch, arch_or_model, loss_func=lf, lr=3e-3,
                    opt_func=opt_func, metrics=metrics)
    else:
        if config is None: config=dict(ps=0., lin_ftrs=[], concat_pool=False)
        learn = cnn_learner(dbch, arch_or_model, pretrained=pretrained, loss_func=lf, lr=3e-3,
                            opt_func=opt_func, metrics=metrics, config=config)
    return learn.to_fp16() if fp16 else learn

In [31]:
# def fit_tune(bs, sz, epochs, lr):
#     learn.dbunch = get_data(bs, sz)
#     do_fit(learn, epochs, lr, freeze=False)

In [32]:
#export
def do_fit(learn, epochs, lr, freeze=False, do_slice=False, **kwargs):
    if do_slice: lr = slice(lr*3,lr)
    if freeze:
        learn.freeze()
        learn.fit_one_cycle(1, lr, div=2, div_final=1, pct_start=0.1)
    learn.unfreeze()
    learn.fit_one_cycle(epochs, lr, **kwargs)

## Submission

In [33]:
#export
def get_test_data(df_tst, bs=512, sz=256, tst_dir='tst_jpg'):
    tst_fns = df_tst.index.values
    tst_splits = [L.range(tst_fns), L.range(tst_fns)]
    tst_dbch = get_data_gen(tst_fns, bs=bs, img_tfm=get_pil_fn(path/tst_dir), sz=sz, splits=tst_splits, test=True)
    tst_dbch.c = 6
    return tst_dbch

In [34]:
#export
def submission(df_tst, preds, fn='submission'):
    ids,labels = [],[]
    for idx,pred in zip(df_tst.index, preds):
        for i,label in enumerate(htypes):
            ids.append(f"{idx}_{label}")
            labels.append('{0:1.10f}'.format(pred[i].item()))
    df_csv = pd.DataFrame({'ID': ids, 'Label': labels})
    df_csv.to_csv(f'{fn}.csv', index=False)
    return df_csv

## Example

In [35]:
name = 'sample'

In [36]:
dbch = get_data(512, 128)
learn = get_learner(dbch, xresnet34)

In [ ]:
do_fit(learn, 1, 1e-2)
learn.save(f'runs/{name}-1')

In [19]:
learn.load(f'runs/{name}-1')

## Example Submission

In [26]:
learn.dbunch = get_test_data(Meta.df_tst, bs=256, sz=256)

In [27]:
# tst = test_dl(learn.dbunch, tst_fns, after_item=[get_pil_fn(path/'tst_jpg'), ToTensor])

# tst.tfms = Pipeline(funcs=[get_pil_fn(path/'tst_jpg'), ToTensor])

In [28]:
sub_fn = f'subm/{name}'

In [ ]:
preds,targs = learn.get_preds()

In [33]:
pred_csv = submission(df_tst, preds, fn=sub_fn)

In [ ]:

FileLink(f'{sub_fn}.csv')

In [7]:
api.competition_submit(f'{nm}.csv', 'testing rsna_retro submission', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:03<00:00, 8.99MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [15]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 13695014,
 'totalBytes': 27277209,
 'date': '2019-12-08T22:49:07.013Z',
 'description': 'testing rsna_retro submission',
 'errorDescription': None,
 'fileName': 'test_sub.csv',
 'publicScore': '0.95475',
 'privateScore': '0.06444',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/13695014/13695014.raw'}

## Save Predictions

In [20]:
#export
class DummyLoss:
    def __call__(self, p, *t, **kwargs): return torch.tensor(0, device=p.device).float()

In [21]:
#export
def save_features(learn, feat_path):
    preds,targs = learn.get_preds(dl=dbunch.valid_dl)
    val_ids = dbunch.valid_dl.dataset.items
    feat_path.mkdir(exist_ok=True)
    for idx,pred in progress_bar(zip(val_ids, preds), total=len(val_ids)):
        np.save(str(feat_path/f'{idx}'), pred.squeeze().numpy())

In [22]:
fn_save = 'runs/baseline_any-3-full'

In [23]:
#export
path_feat256 = path/'features_256'
path_feat256_tst = path/'tst_features_256'

In [25]:
dbunch = get_test_data(Meta.df_comb, bs=512, sz=None, tst_dir='nocrop_jpg256')
learn = get_learner(dbunch, xresnet34, lf=DummyLoss(), metrics=[])
learn.load(fn_save)
learn.model = learn.model[0]
save_features(learn, path_feat256)

In [ ]:
dbunch = get_test_data(Meta.df_tst, bs=512, sz=None, tst_dir='tst_jpg256')
learn = get_learner(dbunch, xresnet34, lf=DummyLoss(), metrics=[])
learn.load(fn_save)
learn.model = learn.model[0]
save_features(learn, path_feat256_tst)

## Export

In [1]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_metadata.ipynb.
Converted 01_preprocess.ipynb.
Converted 02_train.ipynb.
Converted 02_train_01_full.ipynb.
Converted 02_train_01_full_256.ipynb.
Converted 02_train_02_any_resnet18.ipynb.
Converted 02_train_02_any_xresnet18.ipynb.
Converted 02_train_02_any_xresnet34.ipynb.
Converted 02_train_02c_any_old_double_avg_pool.ipynb.
Converted 03_train3d.ipynb.
Converted 03_train3d_00_train3d.ipynb.
Converted 03_train3d_01_train3d-Copy1.ipynb.
Converted 03_train3d_01_train3d-resnet18.ipynb.
Converted 03_train3d_01_train3d.ipynb.
Converted 03_train3d_02_train_head.ipynb.
Converted 04_trainSeq_01_lstm.ipynb.
Converted 04_trainSeq_02_transformer.ipynb.
Converted 05_train_slice.ipynb.
